In [9]:
embedding_size = 300
max_len = 40
vocab_size = 8526

In [14]:
from keras import Input
from keras.layers import Dense,RepeatVector,Embedding,TimeDistributed,LSTM,Concatenate

In [15]:
# 輸入為圖片抽取的特徵
inputs1 = Input(shape=(2048,)) 
# 經過一層全連線層
x1 = Dense(embedding_size, activation='relu')(inputs1)
# 複製max_len次
x1 = RepeatVector(max_len)(x1)

# 輸入部分描述
inputs2 = Input(shape=(max_len,))
# 進行詞嵌入
x2 = Embedding(vocab_size + 1, embedding_size, input_length=max_len, mask_zero=True)(inputs2)
# 經過LSTM層，並保留每個時間步的輸出
x2 = LSTM(256, return_sequences=True)(x2)
# 對每個時間步的輸出，分別應用一層全連線層
x2 = TimeDistributed(Dense(200, activation='relu'))(x2)

# 我們將兩部分處理後的輸入在最後一維進行拼接
x3 = Concatenate()([x1, x2])
# 再經過LSTM，輸出最後一個時間步
x3 = LSTM(256)(x3)
# 將全連線層的結果進行softmax，對映到詞彙表每個詞的概率
outputs = Dense(vocab_size, activation='softmax')(x3)